### Introduction


In [30]:
import gym
import sys
import numpy as np
from collections import defaultdict
from plot_utils import plot_blackjack_values, plot_policy
from tqdm import tqdm

In [2]:
env = gym.make('Blackjack-v0')

/Users/solkim/opt/anaconda3/envs/rl/lib/python3.8/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
print(env.observation_space)
print(env.action_space)

Tuple(Discrete(32), Discrete(11), Discrete(2))
Discrete(2)


In [4]:
print(np.arange(2))

[0 1]


In [13]:
for i_episode in range(10):
    print("GAME START>>>\n")
    state = env.reset()
    while True:
        print("NEXT>>>\n")
        print("current state :",state)
        action = env.action_space.sample()
        print("my action :",action)
        state, reward, done, info = env.step(action)
        print("SOOO,\nnext state : ",state)
        print("reward:",reward)
        print("done? : ",done)
        print("info",info)
        if done:
            print('End game! Reward: ', reward)
            if reward>0:
                print('You won :)\n') 
            elif reward<0: 
                print('You lost :(\n') 
            else :
                print('You draw :|\n')
            break

GAME START>>>

NEXT>>>

current state : (20, 10, False)
my action : 0
SOOO,
next state :  (20, 10, False)
reward: 0.0
done? :  True
info {}
End game! Reward:  0.0
You draw :|

GAME START>>>

NEXT>>>

current state : (9, 10, False)
my action : 1
SOOO,
next state :  (19, 10, False)
reward: 0
done? :  False
info {}
NEXT>>>

current state : (19, 10, False)
my action : 1
SOOO,
next state :  (20, 10, False)
reward: 0
done? :  False
info {}
NEXT>>>

current state : (20, 10, False)
my action : 0
SOOO,
next state :  (20, 10, False)
reward: 1.0
done? :  True
info {}
End game! Reward:  1.0
You won :)

GAME START>>>

NEXT>>>

current state : (13, 10, False)
my action : 1
SOOO,
next state :  (22, 10, False)
reward: -1
done? :  True
info {}
End game! Reward:  -1
You lost :(

GAME START>>>

NEXT>>>

current state : (10, 10, False)
my action : 0
SOOO,
next state :  (10, 10, False)
reward: 1.0
done? :  True
info {}
End game! Reward:  1.0
You won :)

GAME START>>>

NEXT>>>

current state : (6, 6, False)

### Problem 1: MC Prediction

In [6]:
def generate_episode_from_limit_stochastic(bj_env):
    episode = []
    state = bj_env.reset()
    while True:
        probs = [0.8, 0.2] if state[0] > 18 else [0.2, 0.8]
        action = np.random.choice(np.arange(2), p=probs)
        next_state, reward, done, info = bj_env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

In [48]:
for i in range(3): # 경험 누적하기
    ep = generate_episode_from_limit_stochastic(env)
    print(ep)
    s,a,r = zip(*ep)
    print(s,a,r)
    print(len(ep),'\n')

[((20, 10, False), 0, 1.0)]
((20, 10, False),) (0,) (1.0,)
1 

[((18, 3, True), 1, 0), ((15, 3, False), 0, 1.0)]
((18, 3, True), (15, 3, False)) (1, 0) (0, 1.0)
2 

[((21, 10, True), 0, 1.0)]
((21, 10, True),) (0,) (1.0,)
1 



In [53]:
def mc_prediction_q(env, num_episodes, generate_episode, gamma=1.0):
    # Initialize empty dictionaries of arrays
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    for i_episode in tqdm(range(1, num_episodes+1)):
        ep = generate_episode(env)
        T = len(ep)
        
        for t in range(T-1):
            St = ep[t][0]; At = ep[t][1]; Rt = ep[t][2]
            # discounts = np.array([gamma**i for i in range(len(Rt)+1)])
            if t==0:
                N[St][At] += 1
                returns_sum[St][At] += sum(Rt[i:])
                Q[St][At] = returns_sum[St][At]/N[St][At]

    return Q

In [54]:
Q = mc_prediction_q(env, 50000, generate_episode_from_limit_stochastic)
V_to_plot = dict((k,(k[0]>18)*(np.dot([0.8, 0.2],v)) + (k[0]<=18)*(np.dot([0.2, 0.8],v))) \
         for k, v in Q.items())
plot_blackjack_values(V_to_plot)

  0%|          | 0/50000 [00:00<?, ?it/s]


TypeError: 'int' object is not subscriptable

### Problem 2: MC Control

In [23]:
def mc_control(env, num_episodes, alpha, gamma=1.0, eps_start=1.0, eps_decay=.99999, eps_min=0.05):
    nA = env.action_space.n
    Q = defaultdict(lambda: np.zeros(nA))
    epsilon = eps_start
    # loop over episodes
    for i_episode in tqdm(range(1, num_episodes+1)):
        # TODO
        pass

    return policy, Q

In [ ]:
# obtain the estimated optimal policy and action-value function
policy, Q = mc_control(env, 500000, 0.02)

In [ ]:
# obtain the corresponding state-value function
V = dict((k,np.max(v)) for k, v in Q.items())

# plot the state-value function
plot_blackjack_values(V)

In [ ]:
plot_policy(policy)